In [169]:
import os
import sys
from pathlib import Path
import pandas as pd
#import json

%matplotlib inline

In [170]:
sys.path.append(r"../LUCinSA_helpers")
from var_dataframe import *
from rf import *

In [171]:
'''
PARAMETERS: modify in notebook_params notebook, then run that notebook and this cell to update here
DO NOT modify this cell
'''

%store -r basic_config
print(" modelling year is (filter_year): {} (this is first year if season spans two years)".format(basic_config['filter_yr']))
print("sample point file is: {}".format(basic_config['ptfile']))
%store -r timeseries_params
if timeseries_params['load_samp'] == False:
    print('using polygon file to make new point file: {}'.format(basic_config['polyfile']) )
    print('will sample {} pts per polygon'.format(timeseries_params['npts']))
print('first year of calendar sequence is: {} \n'.format(timeseries_params['start_mo']))
%store -r classification_params
print("Classification_Params: \n" 
      " temp output files are saved to (local_model_dir): {} \n" 
      " shared modelling files are in (main_model_dir): {} \n" 
      " feature_model = {} \n "
      " feature_model_dict is: {} \n"
      .format(classification_params['local_model_dir'],classification_params['main_model_dir'],
              classification_params['feature_model'],
              classification_params['feature_mod_dict'] 
              ))

 modelling year is (filter_year): 2021 (this is first year if season spans two years)
sample point file is: /home/downspout-cel/paraguay_lc/vector/sampleData/SamplePts_Dec2023_ALL.csv
first year of calendar sequence is: 7 

Classification_Params: 
 temp output files are saved to (local_model_dir): /home/downspout-cel/paraguay_lc/vector/tests 
 shared modelling files are in (main_model_dir): /home/downspout-cel/paraguay_lc/classification 
 feature_model = Max 
  feature_model_dict is: /home/downspout-cel/paraguay_lc/Feature_Models.json 



### Check feature model settings

## See existing models

In [172]:
with open(classification_params['feature_mod_dict'], 'r+') as feature_model_dict:
    dic = json.load(feature_model_dict)
    models = pd.DataFrame.from_dict(dic, orient='index')
models.head(n=10)

,spec_indices,si_vars,spec_indices_pheno,pheno_vars,singleton_vars,poly_vars,combo_bands,band_names
base6,"[evi2, gcvi, wi, kndvi, nbr, ndmi]","[maxv_yr, minv_yr, amp_yr, avg_yr, sd_yr, cv_y...",[],[],[],[],[],"[evi2_max_yr, evi2_min_yr, evi2_amp_yr, evi2_a..."
Max,"[evi2, gcvi, wi, kndvi, nbr, ndmi]","[maxv_yr, minv_yr, amp_yr, avg_yr, sd_yr, cv_y...",[kndvi],"[maxv_wet, minv_wet, med_wet, slp_wet, numrot_...","[forest_strata, BH, Chaco, Cer, SH]","[pred_ext, pred_dst, pred_area, pred_APR, NovD...",[],"[kndvi_maxv_wet, kndvi_minv_wet, kndvi_med_wet..."
ptsgdb_Max_nomon5-7,None,None,None,None,None,None,"[var_evi2_max_yr, var_evi2_min_yr, var_evi2_am...","[var_gcvi_Nov_20, var_gcvi_Dec_20, var_kndvi_m..."
ptsgdb_Max_no_nbr,None,None,None,None,None,None,"[var_evi2_max_yr, var_evi2_min_yr, var_evi2_am...","[var_gcvi_Nov_20, var_gcvi_Dec_20, var_kndvi_m..."
ptsgdb_Max_no_ndmi,None,None,None,None,None,None,"[var_evi2_max_yr, var_evi2_min_yr, var_evi2_am...","[var_kndvi_eosd_wet, var_evi2_Feb_20, var_wi_m..."


## Load feature model info (if existing) or save feature model info (if new)

##### IF new feature model: Make sure spec_indices, si_vars, singleton_vars and poly_vars are set correctly in parameters (if not, set and rerun parameters cell above)

In [173]:
mod_dict = classification_params['feature_mod_dict']
spec_indices,si_vars,spec_indices_pheno,pheno_vars,singleton_vars,poly_vars,combo_bands,band_names = getset_feature_model(
                      mod_dict, 
                      classification_params['feature_model'], 
                      classification_params['spec_indices'], 
                      classification_params['si_vars'],
                      classification_params['spec_indices_pheno'],
                      classification_params['pheno_vars'],
                      classification_params['singleton_vars'],
                      classification_params['poly_vars']
)
print('Band names: {}'.format(band_names))                                                                      

using existing model: Max 
 spec_indices = ['evi2', 'gcvi', 'wi', 'kndvi', 'nbr', 'ndmi'] 
 si_vars = ['maxv_yr', 'minv_yr', 'amp_yr', 'avg_yr', 'sd_yr', 'cv_yr', 'maxdc_yr', 'mindc_yr', 'Jan_20', 'Feb_20', 'Mar_20', 'Apr_20', 'May_20', 'Jun_20', 'Jul_20', 'Aug_20', 'Sep_20', 'Oct_20', 'Nov_20', 'Dec_20'] 
 pheno_vars = ['maxv_wet', 'minv_wet', 'med_wet', 'slp_wet', 'numrot_wet', 'posd_wet', 'posv_wet', 'numlow_wet', 'tosd_wet', 'p1amp_wet', 'sosd_wet', 'sosv_wet', 'eosd_wet', 'eosv_wet', 'rog_wet', 'ros_wet', 'los_wet'] on ['kndvi'] 
 singleton_vars=['forest_strata', 'BH', 'Chaco', 'Cer', 'SH'] 
 poly_vars = ['pred_ext', 'pred_dst', 'pred_area', 'pred_APR', 'NovDecGCVI_Std']
Band names: ['kndvi_maxv_wet', 'kndvi_minv_wet', 'kndvi_med_wet', 'kndvi_slp_wet', 'kndvi_numrot_wet', 'kndvi_posd_wet', 'kndvi_posv_wet', 'kndvi_numlow_wet', 'kndvi_tosd_wet', 'kndvi_p1amp_wet', 'kndvi_sosd_wet', 'kndvi_sosv_wet', 'kndvi_eosd_wet', 'kndvi_eosv_wet', 'kndvi_rog_wet', 'kndvi_ros_wet', 'kndvi_los_we

## Steps to make variable dataframe -- These steps have already been run - skip to load existing variable dataframe
#### Make variable stack for cells with sample data (Note: This is pretty heavy and should be run from SLURM with bash script (rf0_raster_var_stack))

In [174]:
'''
make_variable_stack(basic_config['smooth_dir'],
                    basic_config['grid_cell'],
                    classification_params['feature_model'],
                    basic_config['yr_range'][0],
                    timeseries_params['start_mo'],
                    classification_params['spec_indices'],
                    classification_params['si_vars'],
                    classification_params['spec_indices_pheno'],
                    classification_params['pheno_vars'],
                    classification_params['feature_mod_dict'],
                    classification_params['singleton_vars'] ,
                    classification_params['singleton_var_dict'],
                    classification_params['poly_vars'], 
                    classification_params['poly_var_path'],
                    classification_params['combo_bands']
                    None)
'''

"\nmake_variable_stack(basic_config['smooth_dir'],\n                    basic_config['grid_cell'],\n                    classification_params['feature_model'],\n                    basic_config['yr_range'][0],\n                    timeseries_params['start_mo'],\n                    classification_params['spec_indices'],\n                    classification_params['si_vars'],\n                    classification_params['spec_indices_pheno'],\n                    classification_params['pheno_vars'],\n                    classification_params['feature_mod_dict'],\n                    classification_params['singleton_vars'] ,\n                    classification_params['singleton_var_dict'],\n                    classification_params['poly_vars'], \n                    classification_params['poly_var_path'],\n                    classification_params['combo_bands']\n                    None)\n"

### Make variable dataframe (use all sample points initially -- can then reduce in Notebook 6b)(Note: best to run through SLURM with bash script (rf1_var_data_frame.sh))

In [175]:
'''
make_var_dataframe(basic_config['smooth_dir'],
                  classification_params['local_model_dir'],
                  basic_config['grid_file'],
                  basic_config['grid_cells'],
                  classification_params['feature_model'],
                  classification_params['feature_mod_dict'],
                  basic_config['yr_range'][0],
                  basic_config['polyfile'],
                  oldest=timeseries_params['oldest_samp'],
                  newest=timeseries_params['newest_samp'],
                  npts=timeseries_params['npts'], 
                  seed=timeseries_params['seed1'],
                  load_samp=timeseries_params['load_samp'], 
                  ptfile=basic_config['ptfile'])
'''

# Alternatively, can append new variables to an existing dataframe without stacking them (much less storage required, but
# will need to stack final set of variables for surface-level classification)

'''
append_feature_dataframe(basic_config['smooth_dir'],
                         ptfile=basic_config['ptfile'],
                         classification_params['samp_pix_vars'],
                         basic_config['grid_cells'],
                         basic_config['grid_file'],
                         classification_params['local_model_dir'],
                         basic_config['yr_range'][0],
                         timeseries_params['start_mo'],
                         classification_params['spec_indices'],
                         classification_params['si_vars'],
                         classification_params['spec_indices_pheno'],
                         classification_params['pheno_vars'],
                         classification_params['singleton_vars'] ,
                         classification_params['singleton_var_dict'],
                         classification_params['poly_vars'], 
                         classification_params['poly_var_path'],
                         classification_params['combo_bands']
''';

## Load existing variable dataframe

In [176]:
pix_vars = pd.read_csv(classification_params['samp_pix_vars'])
pix_vars.head()

,OID_,var_evi2_max_yr,var_evi2_min_yr,var_evi2_amp_yr,var_evi2_avg_yr,var_evi2_sd_yr,var_evi2_cv_yr,var_evi2_Jan_20,var_evi2_Feb_20,var_evi2_Mar_20,...,var_kndvi_numlow_wet,var_kndvi_tosd_wet,var_kndvi_p1amp_wet,var_kndvi_sosd_wet,var_kndvi_sosv_wet,var_kndvi_eosd_wet,var_kndvi_eosv_wet,var_kndvi_rog_wet,var_kndvi_ros_wet,var_kndvi_los_wet
0,25,7118,1214,5904,3657,1840,503,1742,1214,4524,...,3,486,0,283,5968,324,787,-64,-197,41
1,26,7467,1373,6094,3989,1765,442,6636,3024,1933,...,2,425,0,324,1373,425,678,127,-63,101
2,27,4207,1133,3074,2256,1006,446,2214,1746,1550,...,3,456,0,324,789,425,1287,81,-49,101
3,28,7290,1112,6178,3987,2174,545,1787,2586,1754,...,2,425,211,314,987,416,754,109,-118,102
4,29,8287,1237,7050,3693,1947,527,2234,3268,2492,...,2,416,792,305,1036,406,672,115,-116,101


## Remove features from model

In [177]:
new_feature_mod = 'ptsgdb_Max_nopp_nnweg5-9'
drop_indices = ["nbr", "ndmi", "wi", "evi2", "gcvi"] # to drop all variables for an index (e.g. ['nbr'])
drop_vars = ["May_20", "Jun_20", "Jul_20", "Aug_20", "Sept_20"]  # to drop all instances of a given variable (e.g. [`cv_yr`, 'Jun_20'])
drop_combo = ['var_poly_pred_ext', 'var_poly_pred_dst', 'var_poly_pred_area', 'var_poly_pred_APR', 'var_poly_NovDecGCVI_Std'] # to drop specific index_variable combinations (e.g. ['var_wi_Jan_20', 'var_gcvi_maxv_wet'])

#classification_params['samp_pix_vars'] = '/home/downspout-cel/paraguay_lc/vector/ptsgdb_{}.csv'.format(new_feature_mod)
new_feature_mod = 'Max_dropTest2'
drop_indices = [] # to drop all variables for an index (e.g. ['nbr'])
drop_vars = ['Jun_20','Jul_20','Aug_20']  # to drop all instances of a given variable (e.g. [`cv_yr`, 'Jun_20'])
drop_combo = [] # to drop specific index_variable combinations (e.g. ['var_wi_Jan_20', 'var_gcvi_maxv_wet'])

In [178]:
reduce_variable_dataframe(classification_params['samp_pix_vars'], 
                          drop_indices, 
                          drop_vars, 
                          drop_combo, 
                          classification_params['local_model_dir'], 
                          new_feature_mod,  
                          classification_params['feature_mod_dict'])

dropping {'var_gcvi_May_20', 'var_wi_Aug_20', 'var_nbr_Jun_20', 'var_evi2_maxdc_yr', 'var_nbr_Apr_20', 'var_wi_amp_yr', 'var_kndvi_Aug_20', 'var_ndmi_Mar_20', 'var_gcvi_cv_yr', 'var_evi2_Mar_20', 'var_gcvi_Sep_20', 'var_wi_mindc_yr', 'var_ndmi_Feb_20', 'var_ndmi_Oct_20', 'var_nbr_Jul_20', 'var_kndvi_May_20', 'var_evi2_cv_yr', 'var_wi_Feb_20', 'var_ndmi_maxv_yr', 'var_nbr_Dec_20', 'var_ndmi_minv_yr', 'var_nbr_Sep_20', 'var_evi2_Nov_20', 'var_wi_May_20', 'var_nbr_Oct_20', 'var_wi_sd_yr', 'var_wi_Sep_20', 'var_nbr_avg_yr', 'var_evi2_mindc_yr', 'var_wi_Dec_20', 'var_ndmi_maxdc_yr', 'var_gcvi_amp_yr', 'var_gcvi_Jan_20', 'var_gcvi_maxdc_yr', 'var_ndmi_Nov_20', 'var_wi_Nov_20', 'var_evi2_Dec_20', 'var_nbr_cv_yr', 'var_kndvi_Jul_20', 'var_ndmi_May_20', 'var_gcvi_Aug_20', 'var_nbr_mindc_yr', 'var_kndvi_Jun_20', 'var_ndmi_Jul_20', 'var_gcvi_Feb_20', 'var_poly_pred_dst', 'var_evi2_May_20', 'var_ndmi_cv_yr', 'var_poly_pred_area', 'var_wi_maxv_yr', 'var_ndmi_Aug_20', 'var_gcvi_Jun_20', 'var_wi_Apr_

## to augment sample with polygon data: (KW TODO)

* join sample points to polygon data to get subset of polygons with points (join class info to those polygons)
* run make_var_dataframe with load_samp = False and 
    polygon file set to subset above-- will get {npts} random points from each polygon

## To save an html copy of this notebook with all outputs:
(these two cells should be last in notebook)

In [179]:
%%script echo skipping
### comment out above line and run this cell to print output as html

out_name = str(basic_config['country']+'6a_RandomForest_FeathreModel_to_dataframe'+'_model'+str(classification_params['feature_model'])+str(basic_config['filter_yr']))
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --ExecutePreprocessor.store_widget_state=True --output=$out_name 6b_RandomFoest_ModelComparisons.ipynb

skipping
